In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/reddit/submissions.parquet
/kaggle/input/reddit/labeled_post.parquet
/kaggle/input/reddit/labeled_post_genage.parquet


In [2]:
import pandas as pd

In [3]:
df = pd.read_parquet("/kaggle/input/reddit/labeled_post.parquet")

In [4]:
df.shape

(80571, 15)

In [5]:
df.head()

,link_flair_text,num_comments,over_18,score,url,selftext,title,id,edited,is_self,permalink,downs,ups,created,date
0,Not the A-hole,1155,False,9597,https://www.reddit.com/r/AmItheAsshole/comment...,"I (26 F) am married to K (25 M) , i met him a ...",AITA for embarrassing my husband in front of h...,1284npp,0.000000e+00,True,/r/AmItheAsshole/comments/1284npp/aita_for_emb...,0.0,9597,1.680307e+09,2023-04-01
1,Not the A-hole,19,False,8,https://www.reddit.com/r/AmItheAsshole/comment...,Story is just as stupid as the title.\n\nSo my...,AITA from eating a cookie,1284q84,0.000000e+00,True,/r/AmItheAsshole/comments/1284q84/aita_from_ea...,0.0,8,1.680308e+09,2023-04-01
2,Not the A-hole,21,False,13,https://www.reddit.com/r/AmItheAsshole/comment...,My roommate (about to be 27 F) and me (20F) ha...,AITA for yelling at my roommate to stop touchi...,1284sqm,1.680309e+09,True,/r/AmItheAsshole/comments/1284sqm/aita_for_yel...,0.0,13,1.680308e+09,2023-04-01
3,Not the A-hole,13,False,10,https://www.reddit.com/r/AmItheAsshole/comment...,"So as you all know, tomorrow is April Fool’s D...",WIBTA to play an April Fool’s prank on my brot...,1284trz,0.000000e+00,True,/r/AmItheAsshole/comments/1284trz/wibta_to_pla...,0.0,10,1.680308e+09,2023-04-01
4,Asshole,90,False,73,https://www.reddit.com/r/AmItheAsshole/comment...,Using a throwaway because my sister has reddit...,AITA for joking with my sister in front of her...,128509j,0.000000e+00,True,/r/AmItheAsshole/comments/128509j/aita_for_jok...,0.0,73,1.680308e+09,2023-04-01


In [6]:
df.id.nunique(), len(df)

(80571, 80571)

In [7]:
# # https://huggingface.co/finiteautomata/bertweet-base-emotion-analysis

# # Load model directly
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# tokenizer = AutoTokenizer.from_pretrained("finiteautomata/bertweet-base-emotion-analysis")
# model = AutoModelForSequenceClassification.from_pretrained("finiteautomata/bertweet-base-emotion-analysis")

In [10]:
!pip install pysentimiento

In [11]:
# https://github.com/pysentimiento/pysentimiento
from pysentimiento import create_analyzer
analyzer = create_analyzer(task="emotion", lang="en")

2024-05-26 22:43:10.193155: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-26 22:43:10.193260: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-26 22:43:10.346088: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [21]:
results = []

In [22]:
from datetime import datetime

# target_range = range(0, 1000)
last_time = datetime.now()
for i, row in df.iterrows():
#     if not i in target_range:
#         continue

    if i % 1000 == 0:
        now = datetime.now()
        print(datetime.now(), i, (now - last_time).seconds)
        last_time = now
        
    selftext_output = analyzer.predict(row.selftext)
    title_output = analyzer.predict(row.title)

    result = {
        "selftext_output": selftext_output.output,
        "selftext_probas": selftext_output.probas,
        "title_output": title_output.output,
        "title_probas": title_output.probas,
        "row_number": i,
        "id": row.id,
    }
    results.append(result)


2024-05-26 22:46:19.489827 0 0
2024-05-26 22:46:51.244408 1000 31
2024-05-26 22:47:23.279712 2000 32
2024-05-26 22:47:55.598617 3000 32
2024-05-26 22:48:28.092986 4000 32
2024-05-26 22:49:00.884106 5000 32
2024-05-26 22:49:32.931670 6000 32
2024-05-26 22:50:04.887159 7000 31
2024-05-26 22:50:36.877360 8000 31
2024-05-26 22:52:44.350958 12000 31
2024-05-26 22:53:15.970909 13000 31
2024-05-26 22:53:48.023418 14000 32
2024-05-26 22:54:19.900876 15000 31
2024-05-26 22:54:51.960065 16000 32
2024-05-26 22:55:23.870305 17000 31
2024-05-26 22:55:56.087795 18000 32
2024-05-26 22:56:28.344407 19000 32
2024-05-26 22:57:00.334770 20000 31
2024-05-26 22:57:32.501698 21000 32
2024-05-26 22:58:04.912503 22000 32
2024-05-26 22:58:37.425374 23000 32
2024-05-26 22:59:10.226869 24000 32
2024-05-26 22:59:43.214214 25000 32
2024-05-26 23:00:15.963863 26000 32
2024-05-26 23:00:48.798187 27000 32
2024-05-26 23:01:21.702670 28000 32
2024-05-26 23:01:54.566185 29000 32
2024-05-26 23:02:27.900340 30000 33
2024-

In [23]:
df_result = pd.DataFrame(results)

In [30]:
len(df_result)

80571

In [25]:
import json

with open("reddit-sentiment-analysis-pysentimiento.json", "w") as f:
    json.dump(results, f)

In [31]:
df_merge = df.merge(df_result, on='id', how='outer')

In [33]:
df_merge[['id', 'title', 'title_output', 'title_probas', 'selftext', 'selftext_output', 'selftext_probas']]

,id,title,title_output,title_probas,selftext,selftext_output,selftext_probas
0,1284npp,AITA for embarrassing my husband in front of h...,others,"{'others': 0.9625220894813538, 'joy': 0.010048...","I (26 F) am married to K (25 M) , i met him a ...",others,"{'others': 0.964489758014679, 'joy': 0.0243842..."
1,1284q84,AITA from eating a cookie,others,"{'others': 0.9699603319168091, 'joy': 0.015046...",Story is just as stupid as the title.\n\nSo my...,others,"{'others': 0.9357547163963318, 'joy': 0.007309..."
2,1284sqm,AITA for yelling at my roommate to stop touchi...,others,"{'others': 0.9268996715545654, 'joy': 0.027347...",My roommate (about to be 27 F) and me (20F) ha...,disgust,"{'others': 0.18349933624267578, 'joy': 0.00361..."
3,1284trz,WIBTA to play an April Fool’s prank on my brot...,others,"{'others': 0.9651095867156982, 'joy': 0.007270...","So as you all know, tomorrow is April Fool’s D...",others,"{'others': 0.9717631936073303, 'joy': 0.015198..."
4,128509j,AITA for joking with my sister in front of her...,others,"{'others': 0.5893481969833374, 'joy': 0.004741...",Using a throwaway because my sister has reddit...,others,"{'others': 0.9781748056411743, 'joy': 0.008543..."
...,...,...,...,...,...,...,...
80566,18pnieb,AITA- for ignoring my sister because she snitc...,others,"{'others': 0.6990118622779846, 'joy': 0.007435...","f(17), I'm the oldest child with 1 younger sis...",others,"{'others': 0.9589520692825317, 'joy': 0.025777..."
80567,18pno1q,AITA for telling my mom her dog needs training...,others,"{'others': 0.6080728769302368, 'joy': 0.005782...","About a year and a half ago, my (32F) mom (60F...",others,"{'others': 0.9736974239349365, 'joy': 0.013149..."
80568,18pnoev,AITA... I just wanted some peace and quiet for...,others,"{'others': 0.5493558645248413, 'joy': 0.022170...","Throughout the days before my b-day, I was thi...",others,"{'others': 0.9576997756958008, 'joy': 0.028601..."
80569,18pnp6m,AITA for yelling at my boyfriend for eating my...,others,"{'others': 0.9338399767875671, 'joy': 0.031086...",\n\nEdit: Boyfriend insist I add that we've b...,others,"{'others': 0.6862488389015198, 'joy': 0.005734..."


In [36]:
df_merge.to_parquet("labeled_post_sentiment-analysis-pysentimiento.parquet")

In [37]:
df_merge.title_output.value_counts()

title_output
others      64016
disgust     13072
anger        1976
sadness       662
joy           652
fear          145
surprise       48
Name: count, dtype: int64

In [38]:
df_merge.selftext_output.value_counts()

selftext_output
others      68858
disgust      4696
sadness      3177
fear         1780
joy          1695
anger         216
surprise      149
Name: count, dtype: int64